In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#!pip install transformers

In [3]:
#Retrieve pipeline of modules 
from transformers import pipeline
import pandas as pd
import csv
from tqdm import tqdm
import time
from datetime import timedelta

In [4]:
#Choose English to French translation
translator = pipeline("translation_en_to_fr")

No model was supplied, defaulted to t5-base (https://huggingface.co/t5-base)


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [5]:
tqdm.pandas()

In [6]:

#read csv file from github repo
acled = pd.read_csv('https://raw.githubusercontent.com/obengdouglas/DSI-2022/main/module3-nlp-twist/Data/ACLED-2022-03-01-2022-03-31-Western_Africa.csv', sep=",", nrows = 981, skip_blank_lines=True, na_filter=True)
acled.tail()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
976,8893060,132,CPV126,126,01-Mar-22,2022,1,Protests,Peaceful protest,Protesters (Ukraine),...,Praia,14.9315,-23.5125,1,A Nacao; RTC (Cape Verde); Expresso das Ilhas,National,"On 1 March 2022, two nationals from Ukraine an...",0,1646691987,CPV
977,8899603,566,NIG23966,23966,01-Mar-22,2022,1,Battles,Armed clash,Military Forces of Nigeria (2015-),...,Kankara,11.9302,7.4127,2,Blueprint; Sun (Nigeria),National,"On 1 March 2022, Nigerian ground troops with a...",6,1647284179,NGA
978,8899905,854,BFO6764,6764,01-Mar-22,2022,1,Strategic developments,Looting/property destruction,JNIM: Group for Support of Islam and Muslims,...,Soualimou,12.6262,0.4006,1,Undisclosed Source,Local partner-Other,"Looting: On 1 March 2022, presumed JNIM milita...",0,1647284180,BFA
979,8909954,466,MLI6308,6308,01-Mar-22,2022,1,Violence against civilians,Attack,Military Forces of Mali (2021-),...,Massabougou,14.8154,-5.8962,2,Whatsapp; HRW; Reuters; Pinal (Mali); RFI; FAM...,Other-New media,"On 1 March 2022, Malian soldiers killed at lea...",34,1647888805,MLI
980,8968005,566,NIG23911,23911,01-Mar-22,2022,1,Violence against civilians,Attack,Niger Communal Militia (Nigeria),...,Kontagora,10.3999,5.4695,2,Daily Trust (Nigeria),National,"On 1 March 2022, Niger militiamen set the enti...",0,1648594099,NGA


In [7]:
acled.describe()

,data_id,iso,event_id_no_cnty,year,time_precision,inter1,inter2,interaction,latitude,longitude,geo_precision,fatalities,timestamp
count,9.810000e+02,981.000000,981.000000,981.0,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,9.810000e+02
mean,8.932698e+06,599.985729,13306.629969,2022.0,1.122324,3.372069,4.244648,35.102956,11.435517,1.105866,1.391437,3.118247,1.648189e+09
std,4.557605e+04,173.465881,9335.754145,0.0,0.357601,1.812990,2.983506,17.427242,3.249743,7.111876,0.502735,10.954837,8.604886e+05
min,8.891936e+06,132.000000,126.000000,2022.0,1.000000,1.000000,0.000000,10.000000,4.376400,-25.064700,1.000000,0.000000,1.646678e+09
25%,8.899880e+06,466.000000,6413.000000,2022.0,1.000000,2.000000,1.000000,23.000000,9.282300,-2.305000,1.000000,0.000000,1.647284e+09
50%,8.917398e+06,566.000000,6910.000000,2022.0,1.000000,3.000000,6.000000,28.000000,11.873300,1.178700,1.000000,0.000000,1.648494e+09
75%,8.974339e+06,854.000000,24094.000000,2022.0,1.000000,5.000000,7.000000,47.000000,13.790100,6.885400,2.000000,2.000000,1.649088e+09
max,9.106988e+06,854.000000,24503.000000,2022.0,3.000000,8.000000,8.000000,78.000000,20.943400,14.299300,3.000000,130.000000,1.650317e+09


In [8]:
start_time = time.monotonic()

In [9]:
acled["notes_fr"] = acled.notes.progress_apply(translator)
acled.head()

100%|██████████| 981/981 [1:43:16<00:00,  6.32s/it]


,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3,notes_fr
0,8974302,854,BFO6949,6949,31-Mar-22,2022,1,Violence against civilians,Attack,JNIM: Group for Support of Islam and Muslims a...,...,13.6451,-0.7156,2,Facebook; L'Observateur Paalga; Infowakat; Min...,New media-National,"On 31 March 2022, presumed JNIM or IS Sahel mi...",20,1649088162,BFA,"[{'translation_text': 'Le 31 mars 2022, des mi..."
1,8974306,854,BFO6964,6964,31-Mar-22,2022,1,Strategic developments,Looting/property destruction,JNIM: Group for Support of Islam and Muslims,...,11.9333,1.6167,1,Gulmu Info; Facebook; Undisclosed Source,Local partner-Other,"Looting: On 31 March 2022, presumed JNIM milit...",0,1649088162,BFA,[{'translation_text': 'Le pillage : Le 31 mars...
2,8974377,566,NIG24258,24258,31-Mar-22,2022,2,Battles,Armed clash,Boko Haram - Jamaatu Ahli is-Sunnah lid-Dawati...,...,13.6169,13.2644,2,Daily Post (Nigeria); Sahara Reporters; Daily ...,National-Regional,"Around 31 March 2022 (as reported), Boko Haram...",20,1649088162,NGA,[{'translation_text': 'Vers le 31 mars 2022 (t...
3,8974378,566,NIG24259,24259,31-Mar-22,2022,1,Battles,Armed clash,Unidentified Armed Group (Nigeria),...,6.2658,6.5973,2,Nigeria Punch; Guardian (Nigeria),National,"On 31 March 2022, armed individuals attacked s...",1,1649088162,NGA,"[{'translation_text': 'Le 31 mars 2022, des in..."
4,8974400,566,NIG24300,24300,31-Mar-22,2022,1,Strategic developments,Disrupted weapons use,Police Forces of Nigeria (2015-),...,10.5422,7.3302,1,Sahara Reporters; Daily Trust (Nigeria); Niger...,National-Regional,"Defusal: On 31 March 2022, operatives of the a...",0,1649088162,NGA,[{'translation_text': 'Défaut : Le 31 mars 202...


In [10]:
end_time = time.monotonic()
print(timedelta(seconds=end_time - start_time))

1:43:16.600929


In [11]:
acled.to_csv("acled_fr.csv", index = False)

In [12]:
acled.to_csv("C:\Users\obeng\Downloads\acled_fr.csv", index = False)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2780555527.py, line 1)